# 1-D convolution for text analytics
- It is commonly considered that RNNs are for text analysis and CNNs are for image analysis. However, It could be the other way around as well.
- With help of features such as local connectivity, sliding filters, weight sharing, etc., CNNs can be attractive for text analysis as.
- Keras has ```Conv1D``` layer, which is similar to ```Conv2D```
    - ```Conv1D``` is attractive substitute for recurrent models in learning context-based data

In [17]:
from keras.models import *
from keras.layers import *
from keras.datasets import reuters
from keras.preprocessing import sequence
from keras.utils import to_categorical
import matplotlib.pyplot as plt

## ```Conv1D``` layer
- Temporal convolution - "creates a convolution kernel that is convolved with the layer input over a single spatial (or temporal) dimension to produce a tensor of outputs"
    - input: 3D tensor of shape ```(batch_size, steps, input_dim)```
    - output: 3D tensor of shape ```(batch_size, new_steps, filters)```

In [32]:
# setting num_filters = 5 and kernel_size = 3
# 0-10 timesteps & kernel_size = 3 => 8 new steps
conv1d = Conv1D(5, 3, padding = 'valid')(Input(shape = (10, 30)))
print(conv1d.shape)

(?, 8, 5)


In [33]:
# when padding = 'same'
conv1d = Conv1D(5, 3, padding = 'same')(Input(shape = (10, 30)))
print(conv1d.shape)

(?, 10, 5)


In [34]:
# comparison to Conv2D layer
conv2d = Conv2D(5, (3,3), padding = 'valid')(Input(shape = (5, 4, 3)))
print(conv2d.shape)
conv2d = Conv2D(5, (3,3), padding = 'same')(Input(shape = (5, 4, 3)))
print(conv2d.shape)

(?, 3, 2, 5)
(?, 5, 4, 5)


## Using ```Conv1D``` layer in Network

In [2]:
# parameters to import dataset
def get_reuters(num_words = 3000, maxlen = 50):    
    (X_train, y_train), (X_test, y_test) = reuters.load_data(num_words = num_words, maxlen = maxlen)

    X_train = sequence.pad_sequences(X_train, maxlen = maxlen, padding = 'post')
    X_test = sequence.pad_sequences(X_test, maxlen = maxlen, padding = 'post')
    y_train = to_categorical(y_train, num_classes = 46)
    y_test = to_categorical(y_test, num_classes = 46)

    print(X_train.shape)
    print(X_test.shape)
    print(y_train.shape)
    print(y_test.shape)
    
    return X_train, X_test, y_train, y_test

In [3]:
num_words = 3000
max_len = 50
embed_size = 100

In [4]:
X_train, X_test, y_train, y_test = get_reuters(num_words, max_len)

(1595, 50)
(399, 50)
(1595, 46)
(399, 46)


In [38]:
def one_dim_convolution_model(num_words, embed_size, input_length):
    model = Sequential()
    model.add(Embedding(num_words, embed_size, input_length = max_len))
    model.add(Conv1D(50, 10, activation = 'relu'))
    model.add(MaxPooling1D(10))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(46, activation = 'softmax'))
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])
    return model

In [39]:
model = one_dim_convolution_model(num_words, embed_size, max_len)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 100)           300000    
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 41, 50)            50050     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 4, 50)             0         
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 46)                2346      
Total params: 352,396
Trainable params: 352,396
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train, epochs = 100, batch_size = 100)

In [41]:
result = model.evaluate(X_test, y_test)

 32/399 [=>............................] - ETA: 0s

In [42]:
print('Test Accuracy: ', result[1])

Test Accuracy:  0.894736842553


## Diversifying Size of Kernels
- In the previous example, size of kernels were kept constant. However, it is also possible to perform convolution operations with different size of kernels parallel, and merge them afterwards
    - This could be done using ```Functional API``` when creating model

<br>
<img src="http://www.wildml.com/wp-content/uploads/2015/11/Screen-Shot-2015-11-06-at-8.03.47-AM.png" style="width: 600px"/>

In [8]:
num_words = 3000
max_len = 50
embed_size = 100
kernel_sizes = 5, 10, 15

X_train, X_test, y_train, y_test = get_reuters(num_words, max_len)

(1595, 50)
(399, 50)
(1595, 46)
(399, 46)


In [11]:
def one_dim_convolution_model_with_diff_kernels(num_words, embed_size, input_length, kernel_sizes):
    inputs = Input(shape = (X_train.shape[1],))
    embedded = Embedding(output_dim = embed_size, input_dim = num_words, input_length = max_len)(inputs)
    conv_results = []
    for kernel_size in kernel_sizes:
        x = Conv1D(50, kernel_size, activation = 'relu')(embedded)
        x = MaxPooling1D(pool_size = max_len - kernel_size + 1)(x)
        conv_results.append(x)
    conv_result = concatenate(conv_results)
    x = GlobalMaxPooling1D()(conv_result)
    outputs = Dense(46, activation = 'softmax')(x)
    model = Model(inputs = inputs, outputs = outputs)
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])
    return model

In [13]:
model = one_dim_convolution_model_with_diff_kernels(num_words, embed_size, max_len, kernel_sizes)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 50, 100)      300000      input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 46, 50)       25050       embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 41, 50)       50050       embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d_7 (

In [ ]:
history = model.fit(X_train, y_train, epochs = 100, batch_size = 100)

In [15]:
result = model.evaluate(X_test, y_test)

399/399 [==============================] - 0s     


In [16]:
print('Test Accuracy: ', result[1])

Test Accuracy:  0.897243108218
